In [1]:
#Importing libraries
import pandas as pd

In [3]:
#Reading in CSV with mapping from LSOAs to postcodes
dfmap = pd.read_csv('input_data/pcd_lsoa_map.csv', dtype={'pcd': str, 'lsoa11' : str, 'lat' : float, 'long' : float})

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (45) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
#Selecting relevant columns
dfmap = dfmap[["pcd", "lsoa11", "lat", "long"]]

In [5]:
#Extracting short postcode
dfmap["pcd_adjusted"] = dfmap["pcd"].str[0:4]
dfmap["pcd_adjusted"] = dfmap["pcd_adjusted"].str.strip()

In [8]:
#Reading in CSVs with LSOA information
dflsoa1 = pd.read_csv('input_data/lsoa_data_1.csv', header = 0)
dflsoa2 = pd.read_csv('input_data/lsoa_data_2.csv', header = 0)

In [9]:
#Merging CSVs with LSOA information
dflsoa = pd.merge_ordered(dflsoa1, dflsoa2, how = "outer", on = "lsoa11")

In [10]:
#Transforming ages into percentages of total population
for key in dflsoa.columns:
    if "ages_" in key:
        dflsoa[key] = dflsoa[key] / dflsoa["all_ages"]
dflsoa = dflsoa.drop(labels = "all_ages", axis = 1).reset_index(drop=True)

In [11]:
#Transforming household types, ownership types, resident types, and dwelling types into percentages
for key in dflsoa.columns[7:22]:
    dflsoa[key] = dflsoa[key] / 100

In [12]:
#Transforming economic activity into percentages
total_economic = dflsoa['economic_active'] + dflsoa['economic_inactive']
for key in dflsoa.columns[23:28]:
    dflsoa[key] = dflsoa[key] / total_economic

In [13]:
#Transforming educational background, and health into percentages
for key in dflsoa.columns[28:38]:
    dflsoa[key] = dflsoa[key] / 100

In [14]:
#Merging LSOA information with postcode mapping
dflsoapcd = pd.merge_ordered(dfmap, dflsoa, how = "outer", on = "lsoa11")

In [16]:
#Aggregating information per postcode
dflsoapcdagg = dflsoapcd.groupby("pcd_adjusted").mean()
dflsoapcdagg = dflsoapcdagg.reset_index()

In [17]:
#Saving to CSV
dflsoapcdagg.to_csv("output_data/pcd_info.csv")